In [5]:
source_dir1 = '/Users/yoshilab/googledrive/research/30regression/result/08010843'
# source_dir1 = '/Users/yoshilab/googledrive/research/20regression/result/'
# source_dir1 = '/Users/yoshilab/googledrive/research/multiple_regression/all_brain/07311311'
# source_dir1 = '/Volumes/share-2/log/learning/age/result/08060501'

In [2]:
import os
import pandas as pd
import numpy as np
import pickle
from plotly.offline import iplot, init_notebook_mode, plot
import plotly.graph_objs as go

In [3]:
import keras
from keras.models import model_from_json

from nilearn import image as nilimg
import sys

/Users/yoshilab/.pyenv/versions/3.5.1/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.



Using TensorFlow backend.


In [6]:
json_string = open(os.path.join(source_dir1, 'model.json')).read()
model = model_from_json(json_string)

model.load_weights(os.path.join(source_dir1, 'weights.hdf5'))

In [7]:
SLICE_NUM = 185
MAX_VALUE = 1914.090820
data_path = '/Volumes/matlab_share/nifti_img/3DEGIR_SAG'

In [11]:
month_list = os.listdir(data_path)
pat_list_dir = '/Users/yoshilab/PycharmProjects/MRA/structure_data'

In [11]:
predict_list = np.empty((0, 2)).astype(np.float32)

In [13]:
for month in month_list:
    patlist = [patient for patient in os.listdir(os.path.join(data_path, month))]
    pat_id_list = pd.read_csv(os.path.join(pat_list_dir, month,'patient_data.csv'),
                              header=None,
                              index_col=None, names=['dir', 'pid', 'page', 'psex'],
                              dtype={'pid': 'object'})
    for patient in patlist:
        nifti_files = os.listdir(os.path.join(data_path, month, patient))
        raw_file = [file for file in nifti_files if file.startswith('2018') & file.endswith('.nii')][0]
        personal_nifti = nilimg.smooth_img(os.path.join(data_path, month, patient, raw_file), fwhm='fast')
        personal_img = personal_nifti.get_data()
        reshape_img = np.reshape(personal_img, (1, personal_img.shape[0], personal_img.shape[1], personal_img.shape[2]))
        
        if reshape_img.shape[3] < SLICE_NUM:
            train_img = np.append(reshape_img, np.zeros((1, reshape_img.shape[1], reshape_img.shape[2], SLICE_NUM - reshape_img.shape[3])), axis=3)
        elif reshape_img.shape[3] == SLICE_NUM:
            train_img = reshape_img
        else:
            sys.stderr.write('Data Size Error: This Nifti file size is ' + str(reshape_img.shape))
            sys.exit()
        
        personal_age = pat_id_list[pat_id_list.dir == str(patient)]['page'].values[0]
        train_label = [int(personal_age.rstrip('Y'))]
        
        train_img /= MAX_VALUE
        predict_age = model.predict(train_img)
        
        data = np.array([[train_label[0], predict_age[0][0]]])
        predict_list = np.append(predict_list, data, axis=0)
        # print(predict_list)

In [9]:
import plotly.plotly as py
import plotly.graph_objs as go

In [10]:
trace = go.Scatter(
    x=predict_list[:, 0],
    y=predict_list[:, 1],
    mode='markers',
    marker=dict(
        color=("#62ef91"),
        size=10
    )
)

data = [trace]

In [13]:
layout = go.Layout(font=dict(size=24),
                   xaxis=dict(title="Real Age",
                              titlefont=dict(size=24),
                              range=[0, 100]),
                   yaxis=dict(title="Predict Age",
                              titlefont=dict(size=24),
                              range=[0, 100])
                   )

fig = go.Figure(data=data, layout=layout)

In [14]:
plot(fig, filename="./predict_result_cnn_real.html", image_height=750, image_width=1200, image="svg", auto_open=False)

'file:///Users/yoshilab/PycharmProjects/MRA/predict_result_cnn_real.html'

In [15]:
np.savetxt('out.csv', predict_list,delimiter=',')

In [14]:
predict_list

array([[57.        ,  0.58008575],
       [43.        ,  0.57354814],
       [38.        ,  0.53197026],
       [42.        ,  0.55861026],
       [46.        ,  0.55325854],
       [52.        ,  0.56397861],
       [47.        ,  0.59798092],
       [63.        ,  0.54537886],
       [60.        ,  0.57296449],
       [68.        ,  0.54253113],
       [31.        ,  0.55709499],
       [63.        ,  0.59384054],
       [50.        ,  0.63078034],
       [51.        ,  0.57394427],
       [55.        ,  0.53613877],
       [71.        ,  0.61700946],
       [63.        ,  0.58068007],
       [47.        ,  0.55002636],
       [45.        ,  0.56319928],
       [67.        ,  0.61276025],
       [60.        ,  0.54593223],
       [27.        ,  0.55536616],
       [57.        ,  0.54765505],
       [68.        ,  0.57490861],
       [54.        ,  0.57577848],
       [79.        ,  0.5863682 ],
       [65.        ,  0.57210702],
       [43.        ,  0.56047755],
       [69.        ,